In [ ]:
import mediapipe as mp # type: ignore
import numpy as np
from deepface import DeepFace
from sklearn.cluster import DBSCAN
import json

import cv2
from fer import FER

In [3]:
# Função para gerar um nome aleatório para as pessoas
def generate_person_name(person_id):
    return f"person_{person_id}"

# Função para detectar rostos e emoções
def detect_faces_and_emotions(frame, face_cascade, emotion_detector, person_counter):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    faces_data = []

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        emotion = emotion_detector.top_emotion(face)
        person_name = generate_person_name(person_counter)
        
        # Salvando o rosto detectado com nome aleatório
        cv2.imwrite(f"faces/{person_name}.jpg", face)
        
        # Armazenar dados da pessoa
        faces_data.append({
            "name": person_name,
            "emotion": emotion[0],  # A emoção detectada
            "coords": (x, y, w, h)
        })
        
        # Desenhando um retângulo ao redor do rosto
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    return frame, faces_data

# Função para detectar atividades e anomalias
def detect_anomalies(frame, prev_frame):
    # Lógica para detecção de movimento anômalo (ex: diferença entre frames)
    motion_detected = False
    diff = cv2.absdiff(frame, prev_frame)
    gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 50, 255, cv2.THRESH_BINARY)
    
    if cv2.countNonZero(thresh) > 500:  # Movimentos significativos
        motion_detected = True

    return motion_detected

# Função para processar o vídeo e gerar relatório
def process_video(input_video):
    cap = cv2.VideoCapture(input_video)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    emotion_detector = FER()
    person_counter = 1
    persons = {}

    prev_frame = None
    frame_number = 0
    report = {"persons": []}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_number += 1
        # Detecção de rostos e emoções
        frame, faces_data = detect_faces_and_emotions(frame, face_cascade, emotion_detector, person_counter)
        
        # Detecção de anomalias
        if prev_frame is not None:
            motion_detected = detect_anomalies(frame, prev_frame)
            if motion_detected:
                # Marcar anomalia no vídeo
                cv2.putText(frame, 'Anomaly Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Atualizando os dados no relatório
        for person in faces_data:
            name = person["name"]
            if name not in persons:
                persons[name] = {
                    "name": name,
                    "movements": [],
                    "anomalies": [],
                    "emotion": []
                }
            
            # Adicionando movimentos e emoções no relatório
            persons[name]["emotion"].append({
                "frame": frame_number,
                "emotion_name": person["emotion"]
            })
            
            if motion_detected:
                persons[name]["anomalies"].append({
                    "frame": frame_number,
                    "anomalie_name": "Movement Anomaly"
                })
        
        prev_frame = frame.copy()

        # Exibindo o vídeo com as demarcações
        cv2.imshow('Video', frame)

        # Salvando vídeo com as demarcações
        out.write(frame)

        # Pressione 'q' para sair
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Gerar relatório
    report["persons"] = list(persons.values())
    
    # Salvar relatório em um arquivo JSON
    with open("report.json", "w") as f:
        json.dump(report, f, indent=4)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    input_video = "video_entrada.mp4"
    # Definindo o arquivo de saída com demarcações
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (640, 480))
    
    # Processando o vídeo
    process_video(input_video)

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [5]:
# Função para reconhecimento facial e análise de emoções
def analyze_faces(frame, front_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    front_faces = front_cascade.detectMultiScale(gray, 1.3, 5)
    emotions = []
    
    faces = []
    faces.extend(front_faces)  
    # faces.extend(profile_faces)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        
        # Análise de emoções
        analysis = DeepFace.analyze(roi, actions=['emotion'], enforce_detection=False)
        emotions.append(analysis[0]['dominant_emotion'])
        
        # Marcação da face no vídeo
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, analysis[0]['dominant_emotion'], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, faces, emotions

# Função para detecção de anomalias nos movimentos (base simples usando diferença de frames)
def detect_anomalies(prev_frame, curr_frame):
    frame_diff = cv2.absdiff(curr_frame, prev_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 25, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    anomalies = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Área mínima para considerar como anômalo
            anomalies.append(contour)
    
    return anomalies

# Função principal para análise de vídeo
def process_video(input_video_path, output_video_path):
    # Inicializando o detector de rostos (Haar Cascade)
    front_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    
    # Abrindo o vídeo
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Configurando o vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))
    
    # Variáveis para análise
    total_frames = 0
    anomalies_detected = 0
    prev_frame = None
    emotions_data = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        total_frames += 1
        frame_copy = frame.copy()
        
        # 1. Reconhecimento facial e análise de emoções
        frame, faces, emotions = analyze_faces(frame, front_cascade)
    
        # 2. Detecção de anomalias
        if prev_frame is not None:
            anomalies = detect_anomalies(prev_frame, frame)
            anomalies_detected += len(anomalies)
            
            for anomaly in anomalies:
                cv2.drawContours(frame, [anomaly], -1, (0, 0, 255), 2)
        
        # Armazenando os dados das emoções
        emotions_data.extend(emotions)
        
        # Salvando o frame processado no vídeo de saída
        out.write(frame)
        
        prev_frame = frame_copy
    
    # Finalizando o vídeo
    cap.release()
    out.release()
    
    # Gerar relatório
    report = {
        "total_frames": total_frames,
        "anomalies_detected": anomalies_detected,
        "emotions_data": emotions_data
    }
    
    # Salvar o relatório em JSON
    with open('video_analysis_report.json', 'w') as f:
        json.dump(report, f, indent=4)

    print("Processamento completo. Relatório gerado e vídeo de saída salvo.")
    return report

# Execução do código
input_video_path = 'video_analise.mp4'  # Caminho para o vídeo de entrada
output_video_path = 'output_video.avi'  # Caminho para o vídeo de saída

report = process_video(input_video_path, output_video_path)

# Exibir o relatório
print("Relatório:")
print(f"Total de frames analisados: {report['total_frames']}")
print(f"Número de anomalias detectadas: {report['anomalies_detected']}")
print(f"Emoções detectadas: {report['emotions_data']}")

25-02-01 14:10:47 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: C:\Users\mathe\.deepface\weights\facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 35.1MB/s]


Processamento completo. Relatório gerado e vídeo de saída salvo.
Relatório:
Total de frames analisados: 3326
Número de anomalias detectadas: 18884
Emoções detectadas: ['fear', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'fear', 'sad', 'fear', 'sad', 'fear', 'fear', 'fear', 'fear', 'fear', 'happy', 'happy', 'fear', 'neutral', 'fear', 'sad', 'neutral', 'fear', 'fear', 'sad', 'fear', 'fear', 'happy', 'happy', 'fear', 'happy', 'fear', 'happy', 'sad', 'sad', 'happy', 'happy', 'sad', 'happy', 'happy', 'surprise', 'happy', 'fear', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', '

In [6]:
import cv2
import json
import numpy as np
from deepface import DeepFace

# Função para reconhecimento facial e análise de emoções
def analyze_faces(frame, front_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    front_faces = front_cascade.detectMultiScale(gray, 1.3, 5)
    emotions = []
    
    faces = []
    faces.extend(front_faces)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        
        # Análise de emoções
        analysis = DeepFace.analyze(roi, actions=['emotion'], enforce_detection=False)
        emotions.append(analysis[0]['dominant_emotion'])
        
        # Marcação da face no vídeo
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, analysis[0]['dominant_emotion'], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, faces, emotions

# Função para detecção de anomalias nos movimentos
def detect_anomalies(prev_frame, curr_frame):
    frame_diff = cv2.absdiff(curr_frame, prev_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 25, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    anomalies = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            anomalies.append(contour)
    
    return anomalies


# Função para detectar atividades baseada na pose
def detect_activities(video_frame):

    # Função para detectar atividades
    # Inicialização do MediaPipe
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

    activities_detected = []  # Lista para armazenar as atividades detectadas
    current_activity = None    # Variável para armazenar a atividade atual

    # Converte a imagem para o formato RGB, que é necessário para o MediaPipe
    rgb_frame = cv2.cvtColor(video_frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        # Aqui você pode adicionar sua lógica para definir as atividades
        # Por simplicidade, vamos apenas exemplificar com base na quantidade de pontos detectados
        landmarks = results.pose_landmarks.landmark
        activity_confidence = len([lm for lm in landmarks if lm.visibility > 0.5])
        
        # Você pode definir atividades baseadas em posição ou condições de visibilidade
        if activity_confidence > 10:  # Exemplo de threshold
            current_activity = 'Atividade Detectada'  # Simples exemplo
            activities_detected.append(current_activity)

    return activities_detected, current_activity

# Função principal para análise de vídeo
def process_video(input_video_path, output_video_path):
    # Inicializando o detector de rostos
    front_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    
    # Abrindo o vídeo
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Configurando o vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))
    
    # Variáveis para análise
    total_frames = 0
    anomalies_detected = 0
    activities_detected = 0
    prev_frame = None
    emotions_data = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        total_frames += 1
        frame_copy = frame.copy()
        
        # 1. Reconhecimento facial e análise de emoções
        frame, faces, emotions = analyze_faces(frame, front_cascade)
    
        # 2. Detecção de anomalias e atividades
        if prev_frame is not None:
            anomalies = detect_anomalies(prev_frame, frame)
            anomalies_detected += len(anomalies)
            activities, current_activity = detect_activities(frame)
            activities_detected += len(activities)

        # Armazenando os dados das emoções
        emotions_data.extend(emotions)
        
        # Salvando o frame processado no vídeo de saída
        out.write(frame)
        
        prev_frame = frame_copy

 # Finalizando o vídeo
    cap.release()
    out.release()
    
    # Gerar relatório
    report = {
        "total_frames": total_frames,
        "anomalies_detected": anomalies_detected,
        "emotions_data": emotions_data,
        "activities_detected": activities_detected
    }
    
    # Salvar o relatório em JSON
    with open('video_analysis_report.json', 'w') as f:
        json.dump(report, f, indent=4)

    print("Processamento completo. Relatório gerado e vídeo de saída salvo.")
    return report

# Execução do código
input_video_path = 'video_analise.mp4'  # Caminho para o vídeo de entrada
output_video_path = 'output_video.avi'  # Caminho para o vídeo de saída

report = process_video(input_video_path, output_video_path)

# Exibir o relatório
print("Relatório:")
print(f"Total de frames analisados: {report['total_frames']}")
print(f"Número de anomalias detectadas: {report['anomalies_detected']}")
print(f"Emoções detectadas: {report['emotions_data']}")
print(f"Atividades detectadas: {report['activity_detected']}")

: 

In [3]:
import cv2
import json
from deepface import DeepFace
import mediapipe as mp

# Função para reconhecimento facial e análise de emoções
def analyze_faces(frame, front_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    front_faces = front_cascade.detectMultiScale(gray, 1.3, 5)
    emotions = []
    
    faces = []
    faces.extend(front_faces)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        
        # Análise de emoções
        analysis = DeepFace.analyze(roi, actions=['emotion'], enforce_detection=False)
        emotions.append(analysis[0]['dominant_emotion'])
        
        # Marcação da face no vídeo
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, analysis[0]['dominant_emotion'], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, faces, emotions

# Função para detecção de anomalias nos movimentos
def detect_anomalies(prev_frame, curr_frame):
    frame_diff = cv2.absdiff(curr_frame, prev_frame)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 25, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    anomalies = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Ajustar o limite conforme necessário
            anomalies.append(contour)
    
    return anomalies

# Função para detectar atividades baseada na pose
def detect_activities(video_frame):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

    activities_detected = []  # Lista para armazenar as atividades detectadas
    current_activity = "Inativo"  # Inicia como inativo

    # Converte a imagem para o formato RGB que é necessário para o MediaPipe
    rgb_frame = cv2.cvtColor(video_frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        activity_confidence = len([lm for lm in landmarks if lm.visibility > 0.5])
        
        # Definindo atividades baseadas na visibilidade dos landmarks
        if activity_confidence > 10:  # Exemplo de threshold
            current_activity = 'Atividade Detectada'  # Simples exemplo
            activities_detected.append(current_activity)

    return activities_detected, current_activity

# Função principal para análise de vídeo
def process_video(input_video_path, output_video_path):
    # Inicializando o detector de rostos
    front_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    
    # Abrindo o vídeo
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Configurando o vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))
    
    # Variáveis para análise
    total_frames = 0
    anomalies_detected = 0
    activities_detected = 0
    prev_frame = None
    emotions_data = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        total_frames += 1
        frame_copy = frame.copy()
        
        # 1. Reconhecimento facial e análise de emoções
        frame, faces, emotions = analyze_faces(frame, front_cascade)
    
        # 2. Detecção de anomalias e atividades
        if prev_frame is not None:
            anomalies = detect_anomalies(prev_frame, frame)
            # Contar anomalias detectadas
            anomalies_detected += len(anomalies)
            
            # Chamando a função para detectar atividades
            activities, current_activity = detect_activities(frame)
            activities_detected += len(activities)

        # Armazenando os dados das emoções
        emotions_data.extend(emotions)
        
        # Salvando o frame processado no vídeo de saída
        out.write(frame)
        
        # Atualiza o frame anterior
        prev_frame = frame_copy

    # Finalizando o vídeo
    cap.release()
    out.release()
    
    # Gerar relatório
    report = {
        "total_frames": total_frames,
        "anomalies_detected": anomalies_detected,
        "emotions_data": emotions_data,
        "activities_detected": activities_detected
    }
    
    # Salvar o relatório em JSON
    with open('video_analysis_report.json', 'w') as f:
        json.dump(report, f, indent=4)

    print("Processamento completo. Relatório gerado e vídeo de saída salvo.")
    return report

# Execução do código
input_video_path = 'video_analise.mp4'  # Caminho para o vídeo de entrada
output_video_path = 'output_video.avi'  # Caminho para o vídeo de saída

report = process_video(input_video_path, output_video_path)

# Exibir o relatório
print("Relatório:")
print(f"Total de frames analisados: {report['total_frames']}")
print(f"Número de anomalias detectadas: {report['anomalies_detected']}")
print(f"Emoções detectadas: {report['emotions_data']}")
print(f"Número de atividades detectadas: {report['activities_detected']}")

pip install tensorflow --upgrade --force-reinstall

ImportError: Traceback (most recent call last):
  File "c:\Users\mathe\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.